# Walkthrough Challenge 5 - Multi-Agent Chatroom

Duration: 30 minutes

## Overview
- In this challenge, you will be introduced to the concept of agents and multi-agent collaboration.
- You will create a multi-agent chatroom where agents can communicate with each other, to solve a common task.

## Prerequisites

- Please ensure that you have completed the [Setup](../setup/setup.ipynb) before starting this challenge.

### Task 1: Configure and Initialize Semantic Kernel

⚠️ Note: You should have already completed all tasks on the [Setup](../setup/setup.ipynb). If you have not, please go back and complete it now.

#### Step 1: Load Semantic Kernel settings

In this step, we will load the Semantic Kernel settings that we created in the [Setup](../setup/setup.ipynb) notebook.

In [29]:
#r "nuget: Microsoft.SemanticKernel, 1.13.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Abstractions, 1.13.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.13.0-alpha"

#!import ../setup/config/Settings.cs
#!import ../setup/config/Utils.cs

Installed Packages Microsoft.SemanticKernel, 1.13.0 Microsoft.SemanticKernel.Agents.Abstractions, 1.13.0-alpha Microsoft.SemanticKernel.Agents.Core, 1.13.0-alpha

#### Step 2: Initialize Semantic Kernel

In [104]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.TextToImage;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.Agents.Chat;
using System.Threading;
using System.Threading.Tasks;
using System.Collections.Generic;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.DotNet.Interactive;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile("../setup/config/settings.json");

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

### Task 2: Build the agent personas


In [97]:
private const string ProductOwnerName = "ProductOwner";
private const string ProductOwnerInstructions =
    """
    You are a Product Owner responsible for ensuring the acceptance criteria align with the product vision and meet the stakeholders' requirements.
    Your goal is to initiate and validate acceptance criteria that clearly define the conditions for a user story to be considered complete.
    Consider feedback from the Technical Lead, UX Designer, and QA Analyst.
    Make the final decision on whether the acceptance criteria are ready and comprehensive.
    Provide feedback if the acceptance criteria are incomplete or unclear, suggesting improvements where necessary.
    If satisfactory, state that the acceptance criteria are approved. If they are approved, also provide the last version of the acceptance criteria.
    Be concise and to the point.
    """;

private const string TechnicalLeadName = "TechnicalLead";
private const string TechnicalLeadInstructions =
    """
    You are a Technical Lead with expertise in assessing the technical feasibility and implementation details of acceptance criteria.
    Your goal is to review the acceptance criteria for technical completeness, potential challenges, and feasibility within the current architecture.
    Provide feedback on potential technical risks or areas needing clarification, and suggest any improvements to the acceptance criteria.
    Be concise and to the point.
    Only provide a single proposal per response.
    """;

private const string UXDesignerName = "UXDesigner";
private const string UXDesignerInstructions =
    """
    You are a UX Designer focused on ensuring that the acceptance criteria support a positive user experience and adhere to design principles.
    Your goal is to review the acceptance criteria to ensure they account for user interaction, usability, and accessibility.
    Provide feedback on how the acceptance criteria can be improved to enhance the user experience.
    Be concise and to the point.
    Only provide a single proposal per response.
    """;

private const string QAAnalystName = "QAAnalyst";
private const string QAAnalystInstructions =
    """
    You are a QA Analyst responsible for ensuring that the acceptance criteria are clear, detailed, and testable.
    Your goal is to review the acceptance criteria to ensure they are clear, concise, and include specific details that can be used for testing.
    Provide feedback on any missing or unclear acceptance criteria and suggest improvements to make them more testable.
    Be concise and to the point.
    Only provide a single proposal per response.
    """;

private const string BusinessAnalystName = "BusinessAnalyst";
private const string BusinessAnalystInstructions =
    """
    You are a Business Analyst responsible for writing clear, concise, and complete acceptance criteria based on the user story.
    Your goal is to draft acceptance criteria that accurately capture the user needs, business value, and testable conditions.
    Consider feedback and suggestions from the Product Owner, Technical Lead, UX Designer, and QA Analyst to refine and improve the acceptance criteria.
    Ensure the acceptance criteria are well-structured and ready for development.
    Be concise and to the point.
    """;


In [98]:
#pragma warning disable SKEXP0110

Microsoft.SemanticKernel.Agents.ChatCompletionAgent agentProductOwner =
    new()
    {
        Instructions = ProductOwnerInstructions,
        Name = ProductOwnerName,
        Kernel = kernel,
    };

Microsoft.SemanticKernel.Agents.ChatCompletionAgent agentTechnicalLead =
    new()
    {
        Instructions = TechnicalLeadInstructions,
        Name = TechnicalLeadName,
        Kernel = kernel,
    };

Microsoft.SemanticKernel.Agents.ChatCompletionAgent agentUXDesigner =
    new()
    {
        Instructions = UXDesignerInstructions,
        Name = UXDesignerName,
        Kernel = kernel,
    };

Microsoft.SemanticKernel.Agents.ChatCompletionAgent agentQAAnalyst =
    new()
    {
        Instructions = QAAnalystInstructions,
        Name = QAAnalystName,
        Kernel = kernel,
    };

Microsoft.SemanticKernel.Agents.ChatCompletionAgent agentBusinessAnalyst =
    new()
    {
        Instructions = BusinessAnalystInstructions,
        Name = BusinessAnalystName,
        Kernel = kernel,
    };

In [99]:
#pragma warning disable SKEXP0110

private sealed class ApprovalTerminationStrategy : TerminationStrategy
{
    // Terminate when the final message contains the term "approve"
    protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
        => Task.FromResult(history[history.Count - 1].Content?.Contains("approved", StringComparison.OrdinalIgnoreCase) ?? false);
}

In [101]:
KernelFunction terminationFunction =
    KernelFunctionFactory.CreateFromPrompt(
        """
        Determine if the copy has been approved.  If so, respond with a single word: yes

        History:
        {{$history}}
        """);

In [107]:
#pragma warning disable SKEXP0110, SKEXP0001

// Create a nexus for agent interaction.
var chat =
    new AgentGroupChat(agentBusinessAnalyst, agentTechnicalLead, agentUXDesigner, agentQAAnalyst, agentProductOwner)
    {
        ExecutionSettings =
            new()
            {
                // Here a TerminationStrategy subclass is used that will terminate when
                // an assistant message contains the term "approved".
                TerminationStrategy =
                    new KernelFunctionTerminationStrategy(terminationFunction, kernel)
                    {
                        // Only the Product Owner may approve.
                        Agents = [agentProductOwner],
                        // Customer result parser to determine if the response is "yes"
                        ResultParser = (result) => result.GetValue<string>()?.Contains("yes", StringComparison.OrdinalIgnoreCase) ?? false,
                        // The prompt variable name for the history argument.
                        HistoryVariableName = "history",
                        // Limit total number of turns
                        MaximumIterations = 10
                    }
            }
    };

// Invoke chat and display messages.
string input =
"""
**User Story**:
As a regular user, I want to securely log into my account using my username and password, so that I can access personalized features and maintain my privacy.

**Acceptance Criteria**:
1. Users can click a "Login" button on the homepage which directs them to the login page.
2. Users enter their username and password in the provided fields.
3. A 'Login' button submits the credentials, which should be encrypted during transmission.
4. Users are redirected to their personalized dashboard upon successful authentication.
5. An error message, "Incorrect username or password. Please try again," appears for invalid credentials.
6. After three consecutive failed login attempts, the account is locked for 15 minutes to prevent unauthorized access.
7. A 'Forgot Password?' link is available that directs users to a secure password reset page.
8. The login process (from clicking 'Login' to reaching the dashboard) is achieved within 5 seconds under standard conditions.
""";

chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, input));
Console.WriteLine($"# {AuthorRole.User}: '{input}'");

await foreach (var content in chat.InvokeAsync())
{
    Console.WriteLine($"# {content.Role} - {content.AuthorName ?? "*"}: '{content.Content}'");
}

Console.WriteLine($"# IS COMPLETE: {chat.IsComplete}");

# user: '**User Story**:
As a regular user, I want to securely log into my account using my username and password, so that I can access personalized features and maintain my privacy.

**Acceptance Criteria**:
1. Users can click a "Login" button on the homepage which directs them to the login page.
2. Users enter their username and password in the provided fields.
3. A 'Login' button submits the credentials, which should be encrypted during transmission.
4. Users are redirected to their personalized dashboard upon successful authentication.
5. An error message, "Incorrect username or password. Please try again," appears for invalid credentials.
6. After three consecutive failed login attempts, the account is locked for 15 minutes to prevent unauthorized access.
7. A 'Forgot Password?' link is available that directs users to a secure password reset page.
8. The login process (from clicking 'Login' to reaching the dashboard) is achieved within 5 seconds under standard conditions.'
# assis

### Task 2: Create a chatbot like experience, mainting the context of the conversation 

We will start by creating a `ChatHistory` object to maintain the context of the conversation. This object will be used to store the conversation history, namely the system prompts, user inputs and the assistant responses.

The `ChatHistory` object will be initialized with the system prompt.`

In [23]:
var chatHistory = new ChatHistory("You are an historian, expert in the Seven Wonders of the Ancient World. Be concise and informative.");

The `MessageOutputAsync` method will allow us to print the latest message from the chat history.

In [24]:
await Utils.MessageOutputAsync(chatHistory);

system: You are an historian, expert in the Seven Wonders of the Ancient World. Be concise and informative.
------------------------


To interact with the chatbot, we will require the `ChatCompletionService`. This will be used to interact with the Large Language Model to generate responses to the user inputs.

After adding the user input to the chat history, we will use the `ChatCompletionService` to generate a response. This response will be added to the chat history and printed to the user.

In [25]:
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

var executionSettings = new OpenAIPromptExecutionSettings
{
    MaxTokens = 500,
    Temperature = 0.2,
    TopP = 0.5
};

chatHistory.AddUserMessage("Hi, what is the Great Pyramid of Giza?");
await Utils.MessageOutputAsync(chatHistory);

var reply = await chatCompletionService.GetChatMessageContentAsync(
        chatHistory,
        executionSettings: executionSettings,
        kernel: kernel);

chatHistory.Add(reply);
await Utils.MessageOutputAsync(chatHistory);

user: Hi, what is the Great Pyramid of Giza?
------------------------
assistant: The Great Pyramid of Giza, located in Egypt, is the oldest and only surviving monument of the Seven Wonders of the Ancient World. It was constructed around 2580–2560 BC during the reign of Pharaoh Khufu (also known as Cheops). Originally standing at 146.6 meters (481 feet), it was the tallest man-made structure in the world for over 3,800 years. The pyramid served as a tomb and was built using millions of limestone blocks, with some weighing as much as 15 tons. It is part of a larger complex that includes two other major pyramids, smaller pyramids, temples, and the Great Sphinx.
------------------------


Let's add the new message to chat history. You will see that the chat model will use the context of the conversation to generate a response.

Take note that chat history counts towards your token usage, so be mindful of the number of messages you send. Strategies such as counting the number of tokens and truncating the history to only include the last few messages can be used to manage token usage.

In [26]:
chatHistory.AddUserMessage("And do you know who built it?");
await Utils.MessageOutputAsync(chatHistory);

var reply = await chatCompletionService.GetChatMessageContentAsync(
        chatHistory,
        executionSettings: executionSettings,
        kernel: kernel);

chatHistory.Add(reply);
await Utils.MessageOutputAsync(chatHistory);

user: And do you know who built it?
------------------------
assistant: The Great Pyramid of Giza was commissioned by Pharaoh Khufu of the Fourth Dynasty of ancient Egypt. The construction is believed to have been overseen by his vizier, Hemiunu. Contrary to popular belief, the pyramid was not built by slaves but rather by a large workforce of skilled laborers, including stone masons, engineers, and workers, who were likely conscripted for labor as a form of tax payment and were well-fed and housed. Recent discoveries suggest that thousands of laborers worked in shifts to construct the pyramid over a period of approximately 20 years.
------------------------


You can experiment interacting with the chatbot by adding more messages to the chat history and observing the responses.

In [27]:
do
{
    try
    {
        var ask = await InteractiveKernel.GetInputAsync("Ask a question to the assistant: ");
        chatHistory.AddUserMessage(ask);
        await Utils.MessageOutputAsync(chatHistory);

        var reply = await chatCompletionService.GetChatMessageContentAsync(
                chatHistory,
                executionSettings: executionSettings,
                kernel: kernel);

        chatHistory.Add(reply);
        await Utils.MessageOutputAsync(chatHistory);
    }
    catch (Exception)
    {
        // break the loop if the user cancels the input
        break;
    }
} while (true);


Error: Input request cancelled

### Task 3: Use your own data to chat with the model

In some scenarions, you may want the model to respond with specific information based on your own data. For example, you may have internal documentation about an appliance, and the model should respond with information that is specific to that appliance, not from the general knowledge it has been trained on.

This approach is called *Retrieval Augmented Generation* (RAG). In this approach, the model first retrieves relevant information from a knowledge source, and then generates a response based on the retrieved information.

In [28]:
var memorybuilder = new KernelMemoryBuilder();

memorybuilder.WithAzureOpenAITextEmbeddingGeneration(new AzureOpenAIConfig()
{
    Auth = AzureOpenAIConfig.AuthTypes.APIKey,
    APIKey = apiKey,
    APIType = AzureOpenAIConfig.APITypes.EmbeddingGeneration,
    Endpoint = azureEndpoint,
    Deployment = "Text-embedding-ada-002"
});

memorybuilder.WithAzureOpenAITextGeneration(new AzureOpenAIConfig()
{
    Auth = AzureOpenAIConfig.AuthTypes.APIKey,
    APIKey = apiKey,
    APIType = AzureOpenAIConfig.APITypes.ChatCompletion,
    Endpoint = azureEndpoint,
    Deployment = model
});

var memory = memorybuilder.Build();


Let's start by importing a document about Germany Labour Law using the *Semantic Memory* library.

In [29]:

await memory.ImportWebPageAsync(
    "https://web.archive.org/web/20231219085537/https://www.ilo.org/ifpdial/information-resources/national-labour-law-profiles/WCMS_158899/lang--en/index.htm");


After that, we can import the *Semantic Memory* library and use it to add the document to the knowledge base. We will do it via the `MemoryPlugin` class.

In [30]:
using Microsoft.KernelMemory;

var memoryPlugin = kernel.ImportPluginFromObject(new MemoryPlugin(memory));

Let's create a prompt representig the Retrieval Augmented Generation (RAG) approach. The prompt will have an input variable that will represent the user question and will have a function call to the `MemoryPlugin` to retrieve the relevant information from the knowledge base, using the `ask` function.

In [31]:
var skPrompt = """
Question to Memory: {{$input}}

Answer from Memory: {{MemoryPlugin.Ask $input}}

If the answer is empty say 'I don't know' otherwise reply with a preview of the answer,
truncated to 15 words. Prefix with one emoji relevant to the content.
""";

var ragFunction = kernel.CreateFunctionFromPrompt(skPrompt);

And we're ready to use the RAG approach to chat with the model. Let's ask a question about Germany Labour Law and see how the model responds.

In [34]:
var answer = await ragFunction.InvokeAsync(kernel, "How many vacations days do I get in Germany?");

Console.WriteLine(answer);

🏝 In Germany, the statutory minimum leave entitlement is 24 days per calendar year, not counting...


And to test the case when the model does not have the information, let's ask a question about Portugal Labour Law.

In [33]:
var answer = await ragFunction.InvokeAsync(kernel, "How many vacations days do I get in Portugal?");

Console.WriteLine(answer);

🌴 I don't know


You successfully completed challenge 4! 🚀🚀🚀

 **[Home](../../Readme.md)**